In [70]:
import io
import json
import streamlit as st
import pandas as pd
import os
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from googleapiclient.http import MediaFileUpload, MediaIoBaseUpload, MediaIoBaseDownload
from googleapiclient.discovery import build
from google.oauth2 import service_account
import time

  # Replace with your file's ID
#list of articles
article_data_id = "1b7eyqlN3lTapBRYcO1VrXGsj_gBVSxQLIyLCPu3UcG8"
#list of districts and alloted funds
district_data_id = "1lwJL-_KQaOY3VSd2cOeOdiR5QOn8yvX3zp6xNfQJo9U"
# past public data
public_data_id = "1sO08BfwN1gzNs_N7XDq1RnqMgJDDKMdq_nsaNhmjKhs"


inst_data_id = "1dOMubywUqJId2gXHwNWp185L3QmadUnwxyFf0DC9M1s"
public_master_id = "1EdEySmYe6ZJUW16f65_q30nkqfbvDADjcmEkAEJrrL4"
master_data_id = "1ry614-7R4-s0uQcv0zrNeS4O0KAbhVEC67rl5_VllGI"

## Access API from local json
creds = service_account.Credentials.from_service_account_file('/Users/aswathshakthi/PycharmProjects/MNP25/data/mnpdatabase-ca1a93fefdd6.json',
        scopes=['https://www.googleapis.com/auth/drive'])

## Access via streamlit secrets
credentials_dict = json.loads(st.secrets["gcp"]["credentials"])
# creds = service_account.Credentials.from_service_account_info(credentials_dict, scopes=['https://www.googleapis.com/auth/drive'])
drive_service = build('drive', 'v3', credentials=creds)


In [71]:
def read_file(file_id):
    try:
        # Specify the desired MIME type for export (e.g., text/csv for Sheets)
        request = drive_service.files().export_media(
            fileId=file_id,
            mimeType='text/csv'  # Change this based on the file type
        )
        file_stream = io.BytesIO()
        downloader = MediaIoBaseDownload(file_stream, request)

        done = False
        while not done:
            status, done = downloader.next_chunk()
            print(f"Download progress: {int(status.progress() * 100)}%")

        file_stream.seek(0)  # Reset the stream position
        df = pd.read_csv(file_stream)
        return df

    except Exception as e:
        st.error(f"Failed to read file: {e}")

In [72]:
def update_file(file_id, updated_df):
    updated_stream = io.BytesIO()
    updated_df.to_csv(updated_stream, index=False)
    updated_stream.seek(0)

    media = MediaIoBaseUpload(updated_stream, mimetype="text/csv")
    updated_file = drive_service.files().update(
        fileId=file_id,
        media_body=media).execute()

    alert1 = st.success(f"File updated: {updated_file.get('id')}")
    time.sleep(1)
    alert1.empty()

In [73]:
articles_df = read_file(article_data_id)
# Find invalid article names
def valid_articles(one):
    invalid_articles = set(one) - set(articles_df["Articles"])

    # Output result
    if invalid_articles:
        print("These articles are not in the master list:")
        print("\n".join(invalid_articles))
    else:
        print("All articles are valid.")


Download progress: 100%


## District


In [74]:
district_df = read_file(master_data_id)
district_df


Download progress: 100%


,NAME OF THE DISTRICT,REQUESTED ARTICLE,ITEM TYPE,QUANTITY,COST PER UNIT,TOTAL COST,ALLOTTED FUNDS,EXCESS/SHORTAGE,COMMENTS,Beneficiary Type
0,Ariyalur,Agri Battery Sprayer,Article,11,4200.0,46200,190000.0,NaN,No,District
1,Ariyalur,Canon Printer 6030B (USB),Article,1,13900.0,13900,NaN,NaN,No,District
2,Ariyalur,Desktop Computer,Article,1,45500.0,45500,NaN,NaN,No,District
3,Ariyalur,Sewing Machine ORD,Article,16,5200.0,83200,NaN,1200.0,No,District
4,Chengalpattu,Agri Power Sprayer (2 STK),Article,2,11760.0,23520,165000.0,NaN,No,District
...,...,...,...,...,...,...,...,...,...,...
384,Virudhunagar,Ex gratia for Deceased,Aid,1,75000.0,75000,NaN,NaN,Rameswari,District
385,Virudhunagar,Medical Aid,Aid,1,50000.0,50000,NaN,NaN,Karuppasamy,District
386,Virudhunagar,Provision materials to Orphanages,Article,1,200000.0,200000,NaN,0.0,Adhiparasakthi elderly aged Home,District
387,Visakapattinam,Bosch Rotary Hammer GBH 220,Article,1,6372.0,6372,20000.0,NaN,No,District


In [75]:
district_df.duplicated(subset=["NAME OF THE DISTRICT","REQUESTED ARTICLE","QUANTITY","COMMENTS"]).any()

np.False_

In [76]:
valid_articles(district_df["REQUESTED ARTICLE"])

All articles are valid.


In [77]:
district_df["NAME OF THE DISTRICT"].nunique()

62

In [78]:
district_df["REQUESTED ARTICLE"].nunique()

83

In [79]:
district_req = district_df.groupby("REQUESTED ARTICLE",as_index=False)["QUANTITY"].sum()

(district_req[district_req["REQUESTED ARTICLE"]=="AHUJA Radios"]["QUANTITY"].to_list()[0])

1

### Public

In [80]:
public_df = read_file(public_master_id).rename(columns={"Article Name": "REQUESTED ARTICLE","Quantity": "QUANTITY"})
public_df

Download progress: 100%


,App. No.,Name,Aadhar (Without Space),Handicapped (Yes / No),Address,Mobile,REQUESTED ARTICLE,Cost Per Unit,QUANTITY,Total Value,Comments,Beneficiary Type,ITEM TYPE
0,P 001,Rekha Elumalai,629795730911,Yes,"96 Palla St, Thiruvannamalai",9003985442,Handicapped Hand Tricycle,7350,1,7350,No,Public,Article
1,P 002,G.J.Arulmanimaran,379283914465,Yes,"5A, First Cross St,Ambattur, Chennai\n",9444278675,Education Aid,15000,1,15000,Rs.15000 for children,Public,Aid
2,P 003,A.Manikandan,320663697895,No,"390, South St, Kallakurichy",6382220459,Education Aid,15000,1,15000,Rs.15000,Public,Aid
3,P 004,V.Jayavel,548331232628,Yes,"46 Jothi Nagar, Thiruvallur",8668000173,Business Aid,10000,1,10000,Rs.10000,Public,Aid
4,P 005,K.Ganesh,765724849130,No,"Kesavarayanpettai, Melmaruvathur",9655989549,Gents Cycle,5500,1,5500,No,Public,Article
...,...,...,...,...,...,...,...,...,...,...,...,...,...
134,P 135,V.Saranya,349367784825,No,"Melapatty, Madurai",8939219409,Girls Cycle,5200,1,5200,No,Public,Article
135,P 136,Ammu.s,761148632786,No,"Manimangalam, Kancheepuram",8675683781,Renovation Aid,10000,1,10000,No,Public,Aid
136,P 137,"Anjali, Thirumalai",261896284949,No,Manimangalam,9043936660,Renovation Aid,10000,1,10000,No,Public,Aid
137,P 138,Ananthan,934364951810,Yes,"Vikkiravandi, Villupuram",9159583083,Flood Relief Fund,15000,1,15000,No,Public,Project


In [81]:
public_df["REQUESTED ARTICLE"].nunique()

28

In [82]:
valid_articles(public_df["REQUESTED ARTICLE"])

All articles are valid.


In [83]:
public_req = public_df.groupby(["REQUESTED ARTICLE"],as_index=False)["QUANTITY"].sum()
public_req

,REQUESTED ARTICLE,QUANTITY
0,Agri Battery Sprayer,1
1,Bosch Electrician Kit 13 Re,1
2,Business Aid,20
3,Construction Aid,6
4,Education Aid,8
5,Electronic weighing scale,1
6,Ex gratia for Deceased,1
7,Financial Aid,5
8,Flood Relief Fund,2
9,Gents Cycle,5


### Institution

In [84]:
inst = read_file(inst_data_id).rename(columns={"Article Name":"REQUESTED ARTICLE","Quantity": "QUANTITY"})
inst_df = inst[inst["Beneficiary Type"]=="Institution"]
inst_df

Download progress: 100%


,App. No.,Institution Name,REQUESTED ARTICLE,QUANTITY,Cost Per Unit,Total Value,Address,Mobile,Comments,Beneficiary Type,ITEM TYPE
0,I 001,Cheyyur Thasildar Office,Desktop Computer,1,45500,45500,"Thasildar Office, Cheyyur, Chengalpattu",0,needs high config PC for e-seva point,Institution,Article
1,I 002,Adhiparasakthi Institutions,Laptop,81,34000,2754000,Melmaruvathur,044 27529096,No,Institution,Article
2,I 003,Govt School - MKM,Steel Cupboard,2,0,0,Marakanam,9444662668,No,Institution,Article
3,I 004,Govt School - KDPM,Office Table 4X2,5,0,0,Kadapakkam,9442826622,No,Institution,Article
4,I 004,Govt School - KDPM,S Type Chair,5,0,0,Kadapakkam,9442826622,No,Institution,Article
5,I 005,Govt.School - AVPR,Office Table 4X2,5,0,0,"Avanipur, Villupuram",8778403593,No,Institution,Article
6,I 006,Govt.School- NMBR,Steel Cupboard,2,0,0,Nolambur,94442375680,No,Institution,Article
7,I 007,Govt.School - CYR,Steel Cupboard,3,0,0,Cheiyur,8339068395,No,Institution,Article
8,I 008,Anbu Thondu Nilayam,Rice 1000 Kgs,1,0,0,Chengalpattu,9865336966,No,Institution,Article


In [85]:
valid_articles(inst_df["REQUESTED ARTICLE"])

All articles are valid.


In [86]:
inst_df["REQUESTED ARTICLE"].nunique()

6

In [87]:
inst_req = inst_df.groupby(["REQUESTED ARTICLE"],as_index=False)["QUANTITY"].sum()
inst_req

,REQUESTED ARTICLE,QUANTITY
0,Desktop Computer,1
1,Laptop,81
2,Office Table 4X2,10
3,Rice 1000 Kgs,1
4,S Type Chair,5
5,Steel Cupboard,7


### Others

In [88]:
oth_df = inst[inst["Beneficiary Type"]=="Others"]
oth_df

,App. No.,Institution Name,REQUESTED ARTICLE,QUANTITY,Cost Per Unit,Total Value,Address,Mobile,Comments,Beneficiary Type,ITEM TYPE


In [89]:
valid_articles(oth_df["REQUESTED ARTICLE"])

All articles are valid.


In [90]:
oth_df["REQUESTED ARTICLE"].nunique()

0

In [91]:
oth_req = oth_df.groupby(["REQUESTED ARTICLE"],as_index=False)["QUANTITY"].sum()
oth_req

,REQUESTED ARTICLE,QUANTITY


In [92]:
total_qty = pd.concat([district_req, public_req,inst_req,oth_req]).groupby('REQUESTED ARTICLE', as_index=False).sum()
total_qty

,REQUESTED ARTICLE,QUANTITY
0,3 Ltr Table Top Wet Grinder,1
1,5 HP Bore well Pump / DOL Starter,2
2,AHUJA Radios,1
3,Agri Battery Sprayer,114
4,Agri Manual Sprayer,33
...,...,...
87,Wet Grinder 3 Ltrs,2
88,Wet Grinder 5 Ltrs,4
89,Wet Grinder Floor Model 2 Ltr,26
90,Wood Saw Machine (P),2


In [93]:
district_df = read_file(master_data_id)[["REQUESTED ARTICLE","QUANTITY","Beneficiary Type"]]
public_df = read_file(public_master_id).rename(columns={"Article Name": "REQUESTED ARTICLE","Quantity": "QUANTITY"})[["REQUESTED ARTICLE","QUANTITY","Beneficiary Type"]]
inst_df = read_file(inst_data_id).rename(columns={"Article Name":"REQUESTED ARTICLE","Quantity": "QUANTITY"})[["REQUESTED ARTICLE","QUANTITY","Beneficiary Type"]]
final = pd.concat([district_df,public_df,inst_df]).reset_index(drop=True).groupby(["REQUESTED ARTICLE","Beneficiary Type"], as_index=False)["QUANTITY"].sum()
final

Download progress: 100%
Download progress: 100%
Download progress: 100%


,REQUESTED ARTICLE,Beneficiary Type,QUANTITY
0,3 Ltr Table Top Wet Grinder,District,1
1,5 HP Bore well Pump / DOL Starter,District,2
2,AHUJA Radios,District,1
3,Agri Battery Sprayer,District,113
4,Agri Battery Sprayer,Public,1
...,...,...,...
112,Wet Grinder 3 Ltrs,Public,1
113,Wet Grinder 5 Ltrs,District,4
114,Wet Grinder Floor Model 2 Ltr,District,26
115,Wood Saw Machine (P),District,2


In [94]:
print("District QTY",district_df["QUANTITY"].sum())
print("Public QTY",public_df["QUANTITY"].sum())
print("Inst&Oth QTY",inst_df["QUANTITY"].sum())
print("total qty - merge",final["QUANTITY"].sum())
print("total qty - added",district_df["QUANTITY"].sum() + public_df["QUANTITY"].sum()+inst_df["QUANTITY"].sum())

District QTY 13586
Public QTY 140
Inst&Oth QTY 105
total qty - merge 13831
total qty - added 13831


In [95]:
# Pivot the DataFrame to create columns for each Beneficiary Type and total
pivot_df = final.pivot_table(index="REQUESTED ARTICLE",columns="Beneficiary Type",values="QUANTITY",aggfunc="sum",fill_value=0).reset_index()

# Add missing columns for Beneficiary Types and calculate the Total
pivot_df["District"] = pivot_df.get("District", 0)
pivot_df["Public"] = pivot_df.get("Public", 0)
pivot_df["Institution"] = pivot_df.get("Institution", 0)
pivot_df["Others"] = pivot_df.get("Others", 0)

pivot_df["Total"] = pivot_df[["District", "Public", "Institution", "Others"]].sum(axis=1)
pivot_df
ord_req_id = "1ou21kOkXQpL-hoaJ-11av2m7Kwk5hsif65jVOiFaU2Y"
existing_data = read_file(ord_req_id)

# Ensure required columns exist
if "Ordered Quantity" not in existing_data:
    existing_data["Ordered Quantity"] = 0
if "Remaining Quantity" not in existing_data:
    existing_data["Remaining Quantity"] = existing_data["Total"]

# Merge existing data with the new ordered quantity
updated_df = pivot_df.merge(
            existing_data[["REQUESTED ARTICLE", "Ordered Quantity", "Remaining Quantity"]],
            on="REQUESTED ARTICLE",
            how="left",
            suffixes=("", "_old")
        ).fillna({"Ordered Quantity": 0, "Remaining Quantity": lambda x: x["Total"]})

updated_df

Download progress: 100%


,REQUESTED ARTICLE,District,Institution,Public,Others,Total,Ordered Quantity,Remaining Quantity
0,3 Ltr Table Top Wet Grinder,1,0,0,0,1,0,1
1,5 HP Bore well Pump / DOL Starter,2,0,0,0,2,0,2
2,AHUJA Radios,1,0,0,0,1,0,1
3,Agri Battery Sprayer,113,0,1,0,114,0,114
4,Agri Manual Sprayer,33,0,0,0,33,0,33
...,...,...,...,...,...,...,...,...
87,Wet Grinder 3 Ltrs,1,0,1,0,2,0,2
88,Wet Grinder 5 Ltrs,4,0,0,0,4,0,4
89,Wet Grinder Floor Model 2 Ltr,26,0,0,0,26,0,26
90,Wood Saw Machine (P),2,0,0,0,2,0,2


In [96]:
print("District Total",updated_df["District"].sum())
print("Public Total",updated_df["Public"].sum())
print("Institution&oth Total",updated_df["Institution"].sum())


District Total 13586
Public Total 140
Institution&oth Total 105


In [97]:
updated_df["Total"].sum()

np.int64(13831)

### All records Button

In [65]:
district_df = read_file(master_data_id)
district_df["App. No."]=""
cnt = 1
for names in district_df["NAME OF THE DISTRICT"].unique():
    district_df.loc[district_df["NAME OF THE DISTRICT"] == names, "App. No."] = "D " + str(f"{cnt:03}")
    cnt += 1

df1 = district_df[["App. No.","NAME OF THE DISTRICT","REQUESTED ARTICLE","ITEM TYPE","QUANTITY","COST PER UNIT","TOTAL COST","Beneficiary Type","COMMENTS"]]
df1 = df1.rename(columns={"NAME OF THE DISTRICT":"Name","REQUESTED ARTICLE":"Article Name","COST PER UNIT":"Cost Per Unit","QUANTITY":"Quantity","TOTAL COST":"Total Value","COMMENTS":"Comments"})
df1

Download progress: 100%


,App. No.,Name,Article Name,ITEM TYPE,Quantity,Cost Per Unit,Total Value,Beneficiary Type,Comments
0,D 001,Ariyalur,Agri Battery Sprayer,Article,11,4200.0,46200,District,No
1,D 001,Ariyalur,Canon Printer 6030B (USB),Article,1,13900.0,13900,District,No
2,D 001,Ariyalur,Desktop Computer,Article,1,45500.0,45500,District,No
3,D 001,Ariyalur,Sewing Machine ORD,Article,16,5200.0,83200,District,No
4,D 002,Chengalpattu,Agri Power Sprayer (2 STK),Article,2,11760.0,23520,District,No
...,...,...,...,...,...,...,...,...,...
384,D 061,Virudhunagar,Ex gratia for Deceased,Aid,1,75000.0,75000,District,Rameswari
385,D 061,Virudhunagar,Medical Aid,Aid,1,50000.0,50000,District,Karuppasamy
386,D 061,Virudhunagar,Provision materials to Orphanages,Article,1,200000.0,200000,District,Adhiparasakthi elderly aged Home
387,D 062,Visakapattinam,Bosch Rotary Hammer GBH 220,Article,1,6372.0,6372,District,No


In [66]:
public_df = read_file(public_master_id)[["App. No.","Name","Article Name","ITEM TYPE","Quantity","Cost Per Unit","Total Value","Beneficiary Type","Comments"]]
public_df

Download progress: 100%


,App. No.,Name,Article Name,ITEM TYPE,Quantity,Cost Per Unit,Total Value,Beneficiary Type,Comments
0,P 001,Rekha Elumalai,Handicapped Hand Tricycle,Article,1,7350,7350,Public,No
1,P 002,G.J.Arulmanimaran,Education Aid,Aid,1,15000,15000,Public,Rs.15000 for children
2,P 003,A.Manikandan,Education Aid,Aid,1,15000,15000,Public,Rs.15000
3,P 004,V.Jayavel,Business Aid,Aid,1,10000,10000,Public,Rs.10000
4,P 005,K.Ganesh,Gents Cycle,Article,1,5500,5500,Public,No
...,...,...,...,...,...,...,...,...,...
134,P 135,V.Saranya,Girls Cycle,Article,1,5200,5200,Public,No
135,P 136,Ammu.s,Renovation Aid,Aid,1,10000,10000,Public,No
136,P 137,"Anjali, Thirumalai",Renovation Aid,Aid,1,10000,10000,Public,No
137,P 138,Ananthan,Flood Relief Fund,Project,1,15000,15000,Public,No


In [67]:
inst_df = read_file(inst_data_id)[["App. No.","Institution Name","Article Name","ITEM TYPE","Quantity","Cost Per Unit","Total Value","Beneficiary Type","Comments"]]
df3  = inst_df.rename(columns={"Institution Name":"Name"})
df3

Download progress: 100%


,App. No.,Name,Article Name,ITEM TYPE,Quantity,Cost Per Unit,Total Value,Beneficiary Type,Comments
0,I 001,Cheyyur Thasildar Office,Desktop Computer,Article,1,45500,45500,Institution,needs high config PC for e-seva point
1,I 002,Adhiparasakthi Institutions,Laptop,Article,81,34000,2754000,Institution,No
2,I 003,Govt School - MKM,Steel Cupboard,Article,2,0,0,Institution,No
3,I 004,Govt School - KDPM,Office Table 4X2,Article,5,0,0,Institution,No
4,I 004,Govt School - KDPM,S Type Chair,Article,5,0,0,Institution,No
5,I 005,Govt.School - AVPR,Office Table 4X2,Article,5,0,0,Institution,No
6,I 006,Govt.School- NMBR,Steel Cupboard,Article,2,0,0,Institution,No
7,I 007,Govt.School - CYR,Steel Cupboard,Article,3,0,0,Institution,No
8,I 008,Anbu Thondu Nilayam,Rice 1000 Kgs,Article,1,0,0,Institution,No


In [68]:
consold = pd.concat([df1,public_df,df3]).reset_index(drop=True)
consold

,App. No.,Name,Article Name,ITEM TYPE,Quantity,Cost Per Unit,Total Value,Beneficiary Type,Comments
0,D 001,Ariyalur,Agri Battery Sprayer,Article,11,4200.0,46200,District,No
1,D 001,Ariyalur,Canon Printer 6030B (USB),Article,1,13900.0,13900,District,No
2,D 001,Ariyalur,Desktop Computer,Article,1,45500.0,45500,District,No
3,D 001,Ariyalur,Sewing Machine ORD,Article,16,5200.0,83200,District,No
4,D 002,Chengalpattu,Agri Power Sprayer (2 STK),Article,2,11760.0,23520,District,No
...,...,...,...,...,...,...,...,...,...
532,I 004,Govt School - KDPM,S Type Chair,Article,5,0.0,0,Institution,No
533,I 005,Govt.School - AVPR,Office Table 4X2,Article,5,0.0,0,Institution,No
534,I 006,Govt.School- NMBR,Steel Cupboard,Article,2,0.0,0,Institution,No
535,I 007,Govt.School - CYR,Steel Cupboard,Article,3,0.0,0,Institution,No


In [69]:
consold["Quantity"].sum()

np.int64(13831)

In [ ]:
|